In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
chicken_curry_train_dir = '../data/food11/train/chicken_curry'
chicken_curry_test_dir = '../data/food11/test/chicken_curry'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(chicken_curry_train_dir, fname) for fname in os.listdir(chicken_curry_train_dir)]
image_paths += [os.path.join(chicken_curry_test_dir, fname) for fname in os.listdir(chicken_curry_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 1000 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 6

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/chicken_curry\1004867.jpg assigned to cluster 3
../data/food11/train/chicken_curry\1014843.jpg assigned to cluster 3
../data/food11/train/chicken_curry\1022681.jpg assigned to cluster 5
../data/food11/train/chicken_curry\1027623.jpg assigned to cluster 3
../data/food11/train/chicken_curry\1035626.jpg assigned to cluster 0
../data/food11/train/chicken_curry\1040283.jpg assigned to cluster 3
../data/food11/train/chicken_curry\1048226.jpg assigned to cluster 4
../data/food11/train/chicken_curry\1051025.jpg assigned to cluster 0
../data/food11/train/chicken_curry\1052361.jpg assigned to cluster 0
../data/food11/train/chicken_curry\1065322.jpg assigned to cluster 0
../data/food11/train/chicken_curry\1067426.jpg assigned to cluster 3
../data/food11/train/chicken_curry\1068592.jpg assigned to cluster 1
../data/food11/train/chicken_curry\1071242.jpg assigned to cluster 1
../data/food11/train/chicken_curry\1075859.jpg assigned to cluster 2
../data/food11/train/chicken_curry

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(chicken_curry_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(chicken_curry_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(chicken_curry_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(chicken_curry_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
chicken_curry_train_gen = ImageDataGenerator(rescale=1./255)
chicken_curry_test_gen = ImageDataGenerator(rescale=1./255)

chicken_curry_train_generator = chicken_curry_train_gen.flow_from_directory(
    chicken_curry_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

chicken_curry_test_generator = chicken_curry_test_gen.flow_from_directory(
    chicken_curry_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_chicken_curry = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_chicken_curry.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_chicken_curry = base_model_chicken_curry.output
x_chicken_curry = GlobalAveragePooling2D()(x_chicken_curry)
x_chicken_curry = Dense(512, activation='relu')(x_chicken_curry)
chicken_curry_predictions = Dense(num_clusters, activation='softmax')(x_chicken_curry)

# Create and compile the model
chicken_curry_model = Model(inputs=base_model_chicken_curry.input, outputs=chicken_curry_predictions)
chicken_curry_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
chicken_curry_model.fit(
    chicken_curry_train_generator,
    steps_per_epoch=chicken_curry_train_generator.samples // 32,
    validation_data=chicken_curry_test_generator,
    validation_steps=chicken_curry_test_generator.samples // 32,
    epochs=30
)

# Save the model
chicken_curry_model.save('fine_grained_chicken_curry_classifier.h5')

Found 900 images belonging to 6 classes.
Found 100 images belonging to 6 classes.
Epoch 1/30
28/28 [==============================] - 8s 274ms/step - loss: 1.4936 - accuracy: 0.4251 - val_loss: 0.9678 - val_accuracy: 0.6562
Epoch 2/30
28/28 [==============================] - 7s 258ms/step - loss: 0.7304 - accuracy: 0.7834 - val_loss: 0.6661 - val_accuracy: 0.7604
Epoch 3/30
28/28 [==============================] - 7s 249ms/step - loss: 0.4720 - accuracy: 0.8813 - val_loss: 0.4914 - val_accuracy: 0.8542
Epoch 4/30
28/28 [==============================] - 7s 248ms/step - loss: 0.3387 - accuracy: 0.9459 - val_loss: 0.3952 - val_accuracy: 0.8958
Epoch 5/30
28/28 [==============================] - 7s 255ms/step - loss: 0.2662 - accuracy: 0.9631 - val_loss: 0.3763 - val_accuracy: 0.8958
Epoch 6/30
28/28 [==============================] - 7s 260ms/step - loss: 0.2150 - accuracy: 0.9804 - val_loss: 0.3331 - val_accuracy: 0.9062
Epoch 7/30
28/28 [==============================] - 7s 255ms/step 